In [1]:
from IPython.core.display import display, HTML

In [2]:
print("If you want to follow along, checkout this pull request:")
display(HTML("""<a href="https://github.com/yeatmanlab/pyAFQ/pull/289">https://github.com/yeatmanlab/pyAFQ/pull/289</a>"""))
print("To use checkout pull request, run the following commands in your pyAFQ folder:")
print("    git checkout -b 36000-plotly master")
print("    git pull https://github.com/36000/pyAFQ.git plotly")
print("\nThis jupyter notebook script is saved here: ")
display(HTML("""<a href="https://github.com/36000/pyAFQ-scripts/blob/master/plotly/Plotly-demo.ipynb">https://github.com/36000/pyAFQ-scripts/blob/master/plotly/Plotly-demo.ipynb</a>"""))

If you want to follow along, checkout this pull request:


To use checkout pull request, run the following commands in your pyAFQ folder:
    git checkout -b 36000-plotly master
    git pull https://github.com/36000/pyAFQ.git plotly

This jupyter notebook script is saved here: 


In [16]:
# The next few cells pertain to setting up an afq object using prek data from s3
# If you have access to our amazon s3, you can try running this with the sample prek subject
# Otherwise, follow along with the parts of the example that do not require this data

import logging
import s3fs
import os.path as op
import os
import AFQ.api as api
from importlib import reload 

In [29]:
reload(api)
import AFQ.viz_libs.afq_plotly as plotly_afq
reload(plotly_afq)

<module 'AFQ.viz_libs.afq_plotly' from '/home/john/pyAFQ/AFQ/viz_libs/afq_plotly.py'>

In [30]:
# set up AFQ api object
def afq_prek_api(subject):    
    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)  

    fs = s3fs.S3FileSystem()
    
    sub_prefix = f"sub-"
        
    dwi_dir = f'prek-diffusion/derivatives/sub-{subject}/dtiInit_ses-pre'
    dmri_prep_dir = op.join(op.expanduser('~'), 'AFQ_data', 'prek-diffusion/dmri_prep')

    prek_fdata_src = op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.nii.gz")
    prek_fbval_src = op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.bvals")
    prek_fbvec_src = op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.bvecs")
    
    prek_fdata = f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/dwi/dwi_aligned_trilin.nii.gz"
    prek_fbval = f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/dwi/dwi_aligned_trilin.bvals"
    prek_fbvec = f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/dwi/dwi_aligned_trilin.bvecs"
    
    anat_dir = f'prek-diffusion/derivatives/sub-{subject}/freesurfer/mri'

    if not op.exists(prek_fdata):
        fs.get(prek_fdata_src, prek_fdata)
        fs.get(prek_fbval_src, prek_fbval)
        fs.get(prek_fbvec_src, prek_fbvec)

        fs.get(op.join(anat_dir, f"aparc+aseg.nii.gz"), f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/anat/aparc+aseg.nii.gz")

    b0_threshold = 50

    tracking_params = {"n_seeds": 2, "directions": 'det', "stop_threshold": 0.2}
    segmentation_params = {"prob_threshold": 10, "b0_threshold": b0_threshold}
    clean_params = {"clean_rounds": 1}
    
    # uncomment this to download relevant files instead of generating them
    # fs.get("prek-diffusion/derivatives_fa/afq/sub1110/sess", op.join(op.expanduser('~'), 'AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre'), recursive=True)

    myafq = api.AFQ(dmri_prep_dir,
                    dwi_folder="dwi",
                    dwi_file="dwi*", 
                    anat_folder="anat",
                    sub_prefix=sub_prefix,
                    b0_threshold=b0_threshold,
                    viz_library="plotly",

                    tracking_params=tracking_params,
                    segmentation_params=segmentation_params,
                    clean_params=clean_params)

    myafq.export_bundles()
    return myafq

In [31]:
myafq = afq_prek_api(1110) # run on example subject

INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /home/john/AFQ_data/templates 
INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /home/john/AFQ_data/callosum_templates 
INFO:AFQ.api:Saving /home/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-ATR_R_tractography.trk
INFO:AFQ.api:Saving /home/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-ATR_L_tractography.trk
INFO:AFQ.api:Saving /home/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-CGC_R_tractography.trk
INFO:AFQ.api:Saving /home/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-CGC_L_tractography.trk
INF

In [4]:
# after afq api is setup, we can look at plotting

In [25]:
import plotly.graph_objects as go
from plotly.offline import iplot
import numpy as np

In [26]:
fig = go.Figure(data=go.Bar(y=[ord('a'), ord('f'), ord('q')]))
iplot(fig)
# double click to zoom out

In [44]:
x_pts = [10, 12, 15, 18]
y_pts = [5, 1, 1, 2]
z_pts = [10, 10, 8, 7]
# figure = go.Figure()
# figure.add_trace(
#     go.Scatter3d(
#         x=x_pts,
#         y=y_pts,
#         z=z_pts,
#         mode='lines',
#         name='Bundle 1',
#         line=dict(
#             width=1,
#             color='green'
#         )
#     )
# )

In [43]:
x_pts_2 = [8, 9, 14, 16, None, 10, 13, 15, 18]
y_pts_2 = [4, 1, 1, 1, None, 7, 2, 2, 3]
z_pts_2 = [11, 11, 6, 5, None, 10, 11, 8, 6]

# figure.add_trace(
#     go.Scatter3d(
#         x=x_pts_2,
#         y=y_pts_2,
#         z=z_pts_2,
#         mode='lines',
#         name='Bundle 2',
#         line=dict(
#             width=6,
#             color='blue'
#         )
#     )
# )

In [39]:
#myafq.show_inline_bundles() # ~0.5 minutes to run

In [34]:
roi = np.zeros((20, 20, 20))
roi[14, 0:5, 5:11] = 1
roi[13, 1:3, 8:9] = 1
roi[15, 2:4, 7:11] = 1


In [40]:
roi_markers = np.where(roi==1)
# figure.add_trace(
#     go.Scatter3d(
#         x=roi_markers[0],
#         y=roi_markers[1],
#         z=roi_markers[2],
#         name='ROI',
#         marker=dict(color='red'),
#         line=dict(color=f"rgba(0,0,0,0)")
#     )
# )

In [41]:
def my_roi(bundle_name):
    figure = myafq.viz.visualize_roi(myafq.export_rois()[0][bundle_name][0], opacity=0.5, name=f'{bundle_name} ROI 1')
    figure = myafq.viz.visualize_roi(myafq.export_rois()[0][bundle_name][1], opacity=0.5, figure=figure, name=f'{bundle_name} ROI 2')
    figure = myafq.viz.visualize_bundles(
        myafq.get_clean_bundles()[0],
        affine=myafq.get_dwi_affine()[0],
        bundle_dict=myafq.bundle_dict,
        bundle=myafq.bundle_dict[bundle_name]['uid'],
        figure=figure, inline=True)
#my_roi('CST_R')

In [56]:
# This will take time
#myafq.show_interactive_ROIs()

INFO:AFQ.viz:Loading Volume...
INFO:AFQ.viz:Loading tractography...
INFO:AFQ.viz:Generating lines from tractography...

KeyboardInterrupt



In [15]:
# Examples of other uses for sliders (ideas?)
display(HTML("""<a href="https://plotly.com/python/sliders/">Slider Examples</a>"""))

In [42]:
# Example of our current use for sliders:
#myafq.viz.visualize_volume(myafq.get_dti_fa()[0], inline=True, which_plane='coronal')

In [16]:
# Examples of other possibly useful plotly graphs
display(HTML("""<a href="https://plotly.com/python/network-graphs/">Network Graphs</a>"""))
display(HTML("""<a href="https://plotly.com/python/v3/continuous-error-bars/">Continuous Error Bars</a>"""))


In [48]:
reload(plotly_afq)
anatomy = op.join(op.expanduser('~'), 'AFQ_data/prek-diffusion/dmri_prep/sub-1110/dtiInit_ses-pre/t1_acpc.nii.gz')
myafq.viz_bundles(volume=anatomy, xform_volume=True, color_by_volume='dti_fa', interactive=True)

INFO:AFQ.viz:Loading Volume...
INFO:AFQ.viz:Loading tractography...
INFO:AFQ.viz:Loading Volume...
INFO:AFQ.viz:Generating lines from tractography...


ValueError: Invalid property specified for object of type plotly.graph_objs.scatter3d.Line: 'opacity'

    Valid properties:
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `line.colorscale`. Has an effect only if in
            `line.color`is set to a numerical array. In case
            `colorscale` is unspecified or `autocolorscale` is
            true, the default  palette will be chosen according to
            whether numbers in the `color` array are all positive,
            all negative or mixed.
        cauto
            Determines whether or not the color domain is computed
            with respect to the input data (here in `line.color`)
            or the bounds set in `line.cmin` and `line.cmax`  Has
            an effect only if in `line.color`is set to a numerical
            array. Defaults to `false` when `line.cmin` and
            `line.cmax` are se

In [45]:
#myafq.viz_ROIs(color_by_volume='dti_fa', export_as_html=True, bundle_names=["ATR_L"])
myafq.viz_bundles(volume=anatomy, xform_volume=True, color_by_volume='dti_fa', export_as_html=True)

INFO:AFQ.viz:Loading Volume...
INFO:AFQ.viz:Loading tractography...
INFO:AFQ.viz:Loading Volume...
INFO:AFQ.viz:Generating lines from tractography...


KeyboardInterrupt: 